**Ομάδα 23**

Νοταράς Ευάγγελος\
10084\
enotaras@ece.auth.gr\
\
Καλλιμάνης Ιωάννης\
10007\
ikallima@ece.auth.gr

In [1]:
from google.colab import files

# Upload the datasetC file
uploaded = files.upload()

Saving datasetC.csv to datasetC.csv


In [2]:
from google.colab import files

# Upload the datasetCTest file
uploaded = files.upload()

Saving datasetCTest.csv to datasetCTest.csv


In [3]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from datetime import datetime

# Testing

Load Dataset

In [ ]:
file_path_train = 'datasetC.csv'

df = pd.read_csv(file_path_train,  header=None, dtype='float64')

X = df.iloc[:, :-1].values  # Features
y = df.iloc[:, -1]   # Labels
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=True, random_state=42)

Προσέγγιση 1: RBF SVM

---



Tuning RBF SVM

In [ ]:
C_values = np.linspace(0.1, 100, 50)
gamma_values = ['scale', 'auto']
param_grid = dict(gamma=gamma_values, C=C_values)


grid = GridSearchCV(SVC(kernel='rbf'), param_grid,  refit=True, verbose=2, return_train_score=True)
grid.fit(X_train, y_train)
print('Best parameters after grid-search:')
print(grid.best_params_)
print('Best mean CV score: ', grid.best_score_)

Δοκιμή στο tuned RBF SVM

In [ ]:
svm_lin_model = SVC(kernel='rbf', C=10, gamma='scale')
svm_lin_model.fit(X_train, y_train)
y_pred = svm_lin_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print('RBF SVM')
print("Accuracy:", accuracy)

Προσέγγιση 2: Random Forests

---



In [ ]:
from sklearn.ensemble import RandomForestClassifier
"""
maxdepth={'max_depth': [5,10, 15,20,25],
          'min_samples_split': [1, 2, 3, 4, 5, 6],
          'min_samples_leaf': [1, 2, 3, 4, 5, 6]}
grid = GridSearchCV(RandomForestClassifier(n_estimators=300), maxdepth,  refit=True, verbose=2, return_train_score=True)
grid.fit(X_train, y_train)
print('Best parameters after grid-search:')
print(grid.best_params_)
print('Best mean CV score: ', grid.best_score_)
"""


In [ ]:
rf_classifier = RandomForestClassifier(n_estimators=500, max_depth=25, min_samples_leaf=5, min_samples_split=2, random_state=42)
rf_classifier.fit(X_train, y_train)

y_pred = rf_classifier.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print('Random Forests')
print("Accuracy:", accuracy)

Προσέγγιση 3: ΚΝΝ

---



In [ ]:
for k in range(5, 30):
    print("k=", k)
    model = KNeighborsClassifier(n_neighbors=k)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    print("Accuracy:", accuracy)

Dimention Reduction with Random Forest

---



Εκτυπώνουμε την σημαντικότητα σύμφωνα με το random forest

In [ ]:
model = RandomForestClassifier()
model.fit(X_train, y_train)

# Access feature importances
feature_importances = model.feature_importances_

# Create a dataframe to display feature importances
feature_importance_df = pd.DataFrame({'Feature': range(len(X[0])), 'Importance': feature_importances})

# Sort features by importance in descending order
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

# Plot feature importances
plt.figure(figsize=(10, 6))
plt.bar(range(len(feature_importances)), feature_importances, tick_label=range(len(X[0])))
plt.xlabel('Features')
plt.ylabel('Importance')
plt.title('Feature Importances from Random Forest')
plt.xticks(rotation=90)
plt.show()

Με βάση το παραπάνω διάγραμμα, ρυθμίζω το thresshold για να μειώσω τις διαστάσεις

In [ ]:
from sklearn.feature_selection import SelectFromModel

threshold = 0.0018
sfm = SelectFromModel(model, threshold=threshold)

X_train_selected = sfm.transform(X_train)
X_test_selected = sfm.transform(X_test)
print("Before reduction")
print(np.shape(X_train))
print("After reduction")
print(np.shape(X_train_selected))

RBF SVM με dimention reduction

---



In [ ]:
C_values = np.linspace(0.1, 50, 100)
gamma_values = ['scale', 'auto']
param_grid = dict(gamma=gamma_values, C=C_values)


grid = GridSearchCV(SVC(kernel='rbf'), param_grid,  refit=True, cv=8, verbose=2, return_train_score=True)
grid.fit(X_train_selected, y_train)
print('Best parameters after grid-search:')
print(grid.best_params_)
print('Best mean CV score: ', grid.best_score_)

In [ ]:
svm_rbf_model = SVC(kernel='rbf', C=10, gamma='scale')
svm_rbf_model.fit(X_train_selected, y_train)
y_pred = svm_rbf_model.predict(X_test_selected)
accuracy = accuracy_score(y_test, y_pred)
print('RBF SVM')
print("Accuracy:", accuracy)

# Final Model

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import LabelEncoder
from torchvision import models, transforms
from torchvision.models import vision_transformer
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F

Load Dataset

In [5]:
dataset_path = "datasetC.csv"
dataset_path_test = 'datasetCTest.csv'

df = pd.read_csv(dataset_path, header=None)
df_test = pd.read_csv(dataset_path_test, header=None)

In [6]:
X_train = df.iloc[:, :-1].values  # Features
y_train = df.iloc[:, -1].values   # Labels

X_test = df_test.values  # Test Features

Split Data and create Dataloaders

In [7]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=16)

label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
#y_test_encoded = label_encoder.transform(y_test)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train_encoded).to(device)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
#y_test_tensor = torch.tensor(y_test_encoded).to(device)


input_size = 20
X_train_tensor = X_train_tensor.view(-1, 1, input_size, input_size)
X_test_tensor = X_test_tensor.view(-1, 1, input_size, input_size)

num_classes = len(np.unique(y_train))

cuda


In [8]:
train_loader = DataLoader(TensorDataset(X_train_tensor, y_train_tensor), batch_size=32, shuffle=True)
#test_loader = DataLoader(TensorDataset(X_test_tensor, y_test_tensor), batch_size=32, shuffle=False)
test_loader = DataLoader(TensorDataset(X_test_tensor), batch_size=32, shuffle=False)

The CNN model that is going to be used

In [9]:
class SimpleCNN(nn.Module):
    def __init__(self, hidden_size1, hidden_size2, hidden_size3, drop1, drop2):
        """
        CNN Model Implementation

        Parameters:
        - hidden_size1, hidden_size2, hidden_size3 (int): Number of neurons in the hidden layers.
        - drop1, drop2 (float): Dropout rate in the hidden layers
        """
        super(SimpleCNN, self).__init__()

        self.features = nn.Sequential(
          nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1),
          nn.ReLU(),
          nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size=2, stride=2),

          nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
          nn.ReLU(),
          nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
          nn.ReLU(),
          nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1),
          nn.ReLU(),
          nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.dense_layers = nn.Sequential(
            nn.Dropout(drop1),
            nn.Linear(3200, hidden_size1),
            nn.BatchNorm1d(hidden_size1),
            nn.ReLU(),
            nn.Dropout(drop2),
            nn.Linear(hidden_size1, hidden_size2),
            nn.BatchNorm1d(hidden_size2),
            nn.ReLU(),
            nn.Linear(hidden_size2, hidden_size3),
            nn.BatchNorm1d(hidden_size3),
            nn.ReLU(),
            nn.Linear(hidden_size3, 5)
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.dense_layers(x)
        return x

In [ ]:
# Define model
model = SimpleCNN(hidden_size1=256, hidden_size2=512, hidden_size3=128, drop1=0.2, drop2=0.2)
model = model.to(device)

# Define optimizer, loss function, and learning rate scheduler
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5, factor=0.2, min_lr=1e-7, verbose=False)

# Training Process

In [10]:
def batch_gd(model, criterion, optimizer, scheduler, train_loader, test_loader=None, n_epochs=30, device='cuda'):
    # Lists to store training and testing losses, and training and testing accuracies for each epoch
    train_losses = []
    test_losses = []
    train_accuracies = []
    test_accuracies = []

    for epoch in range(n_epochs):
        # Set the model to training mode
        model.train()
        total_train_loss = 0.0
        correct_train = 0
        total_train = 0

        # Training loop
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward(retain_graph=True)
            optimizer.step()

            total_train_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total_train += labels.size(0)
            correct_train += (predicted == labels).sum().item()

        # Calculate training accuracy and store results
        train_accuracy = correct_train / total_train
        train_losses.append(total_train_loss / len(train_loader))
        train_accuracies.append(train_accuracy)

        if test_loader is not None:
            # Set the model to evaluation mode
            model.eval()
            with torch.no_grad():
                correct = 0
                total = 0
                total_test_loss = 0.0
                correct_test = 0
                total_test = 0

                # Testing loop
                for inputs, labels in test_loader:
                    inputs, labels = inputs.to(device), labels.to(device)
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    total_test_loss += loss.item()


                    _, predicted = torch.max(outputs, 1)

                    # Handle one-hot encoded labels
                    if len(labels.size()) > 1 and labels.size(1) > 1:
                        labels = torch.argmax(labels, dim=1)

                    total_test += labels.size(0)
                    correct_test += (predicted == labels).sum().item()

                # Calculate testing accuracy and store results
                test_accuracy = correct_test / total_test
                test_losses.append(total_test_loss / len(test_loader))
                test_accuracies.append(test_accuracy)

                # Print test accuracy at the end of every 5 epochs
                if (epoch + 1) % 5 == 0:
                    print(f'Epoch {epoch + 1}/{n_epochs} - Test Accuracy: {test_accuracy:.4f}')

        scheduler.step(train_accuracy)


    return train_losses, test_losses, train_accuracies, test_accuracies

Η παρακάτω συνάρτηση χρησιμοποιήθηκε για cross-validation του μοντέλου κατά το tuning




In [11]:
def custom_cv(X, y, device, hid1, hid2, hid3, drop1, drop2, splits=5):
    # Shuffle the data indices
    np.random.seed(42)
    indices = np.random.permutation(len(X))
    X = X[indices]
    y = y[indices]

    # Lists to store test accuracies for each fold
    test_accuracies=[]

    # Calculate fold size
    fold_size = len(X) // splits

    for fold in range(splits):
        print(f"Fold {fold + 1}/{splits}")

        # Calculate start and end indices for the current fold
        start_idx = fold * fold_size
        end_idx = start_idx + fold_size

        X_train, X_test = np.concatenate([X[:start_idx], X[end_idx:]]), X[start_idx:end_idx]
        y_train, y_test = np.concatenate([y[:start_idx], y[end_idx:]]), y[start_idx:end_idx]

        label_encoder = LabelEncoder()
        y_train_encoded = label_encoder.fit_transform(y_train)
        y_test_encoded = label_encoder.transform(y_test)

        # Convert data to PyTorch tensors and create the dataLoaders
        X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
        y_train_tensor = torch.tensor(y_train_encoded).to(device)
        X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
        y_test_tensor = torch.tensor(y_test_encoded).to(device)

        input_size = 20
        X_train_tensor = X_train_tensor.view(-1, 1, input_size, input_size)
        X_test_tensor = X_test_tensor.view(-1, 1, input_size, input_size)

        num_classes = len(np.unique(y_train))
        train_loader = DataLoader(TensorDataset(X_train_tensor, y_train_tensor), batch_size=32, shuffle=True)
        test_loader = DataLoader(TensorDataset(X_test_tensor, y_test_tensor), batch_size=32, shuffle=False)

        # Initialize the CNN model
        model = SimpleCNN(hidden_size1=hid1, hidden_size2=hid2, hidden_size3=hid3, drop1=drop1, drop2=drop2)
        model=model.to(device)

        optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
        criterion = nn.CrossEntropyLoss()
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=3, factor=0.1)

        # Train the model using batch_gd function
        batch_gd(model, criterion, optimizer, scheduler, train_loader, test_loader, n_epochs=30, device=device)
        model.eval()

        # Evaluate the model on the test set
        test_accuracy=[]
        with torch.no_grad():
          correct = 0
          total = 0

          for inputs, labels in test_loader:
              inputs, labels = inputs.to(device), labels.to(device)

              if len(labels.size()) > 1 and labels.size(1) > 1:
                  labels = torch.argmax(labels, dim=1)

              outputs = model(inputs)
              _, predicted = torch.max(outputs, 1)

              total += labels.size(0)
              correct += (predicted == labels).sum().item()

          test_accuracy.append(correct / total)

          test_accuracies.append(np.mean(test_accuracy, axis=0))

    return test_accuracies

# Results

Εκπαίδευση του μοντέλου, προβολή διαγραμμάτων και έλεγχος στο test set

In [ ]:
t0 = datetime.now()
train_losses, test_losses, train_accuracies, test_accuracies = batch_gd(model, criterion, optimizer, scheduler, train_loader, test_loader, n_epochs=45, device=device)
print(f"Elapsed Time: {(datetime.now() - t0).total_seconds():.3f} seconds")

#Plot the model loss and accuracy in every epoch
plt.figure(figsize=(10, 5))
#Losses
plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Train Loss')
plt.plot(test_losses, label='Test Loss')
plt.legend()
plt.title('Losses')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.grid(True)
#Accuracy
plt.subplot(1, 2, 2)
plt.plot(train_accuracies, label='Train Accuracy')
plt.plot(test_accuracies, label='Test Accuracy')
plt.legend()
plt.title('Accuracies')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.grid(True)
plt.show()

model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    test_accuracy = correct / total
    print(f'Total Accuracy: {test_accuracy:.4f}')

Ακρίβεια με Cross-Validation

In [ ]:
test_accuracies=custom_cv(X, y, device=device, splits=20, hid1=256, hid2=512, hid3=128, drop1=0.2, drop2=0.2)
print("Cross-Validation Accuracy: ", np.mean(test_accuracies))

Γίνεται tuning στο μοντέλο για να επιλεγούν οι κατάλληλοι υπερπαράμετροι

In [ ]:
hidden1=[128,256, 512, 1024]
hidden2=[128,256, 512, 1024]
hidden3=[128, 256, 512, 1024]
dropout1=[0.2, 0.3, 0.4]
dropout2=[0.0, 0.1, 0.2, 0.3]
for hid1 in hidden1:
  for hid2 in hidden2:
    for hid3 in hidden3:
      for drop1 in dropout1:
        for drop2 in dropout2:
          print(hid1, hid2, hid3, drop1, drop2)
          test_accuracies=custom_cv(X, y, device=device, splits=6, hid1=hid1, hid2=hid2, hid3=hid3, drop1=drop1, drop2=drop2)
          print(np.mean(test_accuracies))

# Ensemble Model

In [12]:
def majority_vote(predictions, num_classes=5):
    # Initialize vote counts for each class
    class_votes = [0] * num_classes

    # Count votes from each model
    for prediction in predictions:
        predicted_class = np.argmax(prediction)
        class_votes[predicted_class] += 1

    # Select the class with the most votes
    final_prediction = np.argmax(class_votes)
    return final_prediction

In [14]:
t0=datetime.now()
models = []

# Training 30 models
for i in range(30):
    print("----------Model", (i+1), "----------")

    model = SimpleCNN(hidden_size1=256, hidden_size2=512, hidden_size3=128, drop1=0.2, drop2=0.2)  # Create and compile your CNN model
    model=model.to(device)

    optimizer = optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5, factor=0.2, min_lr=1e-7, verbose=False)

    batch_gd(model, criterion, optimizer, scheduler, train_loader, n_epochs=45, device=device)

    models.append(model)
    model.eval()
print(f"Elapsed Time: {(datetime.now() - t0).total_seconds():.3f} seconds")


----------Model 1 ----------
----------Model 2 ----------
----------Model 3 ----------
----------Model 4 ----------
----------Model 5 ----------
----------Model 6 ----------
----------Model 7 ----------
----------Model 8 ----------
----------Model 9 ----------
----------Model 10 ----------
----------Model 11 ----------
----------Model 12 ----------
----------Model 13 ----------
----------Model 14 ----------
----------Model 15 ----------
----------Model 16 ----------
----------Model 17 ----------
----------Model 18 ----------
----------Model 19 ----------
----------Model 20 ----------
----------Model 21 ----------
----------Model 22 ----------
----------Model 23 ----------
----------Model 24 ----------
----------Model 25 ----------
----------Model 26 ----------
----------Model 27 ----------
----------Model 28 ----------
----------Model 29 ----------
----------Model 30 ----------
Elapsed Time: 1153.796 seconds


Για να ελέγξουμε το συνενωμένο μοντέλο με train-test split

In [ ]:
all_predictions = []

# Make predictions using each model
with torch.no_grad():
    for model in models:
        model_predictions = []
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            model_predictions.extend(predicted.cpu().numpy())  # Store predictions as a list
        all_predictions.append(model_predictions)

# Transpose the predictions to have one row per sample
all_predictions = list(map(list, zip(*all_predictions)))
final_predictions = [max(set(prediction), key=prediction.count) for prediction in all_predictions]

# Calculate overall accuracy
correct = sum(final_pred == labels.item() for final_pred, labels in zip(final_predictions, y_test_encoded))
total = len(y_test_encoded)
test_accuracy = correct / total

print(f'Total Accuracy: {test_accuracy:.4f}')

Total Accuracy: 0.9540


In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

conf_mat = confusion_matrix(y_test_encoded, final_predictions)

# Plot the confusion matrix using seaborn
plt.figure(figsize=(10, 8))
sns.heatmap(conf_mat, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

Για να σώσουμε το numpy αρχείο με τις προβλεπόμενες ετικέτες

In [26]:
all_predictions = []

# Make predictions using each model for the test set without labels
with torch.no_grad():
    for model in models:
        model_predictions = []
        for inputs in test_loader:
            inputs = inputs[0].to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            model_predictions.extend(predicted.cpu().numpy())  # Store predictions as a list
        all_predictions.append(model_predictions)

# Transpose the predictions to have one row per sample
all_predictions = list(map(list, zip(*all_predictions)))

# Create the final predictions for the test set without labels
final_predictions = [max(set(prediction), key=prediction.count) for prediction in all_predictions]
final_predictions = [pred + 1 for pred in final_predictions]
# Save the predictions to a file (labels23.npy)
np.save('labels23.npy', final_predictions)
print("Predictions saved to labels23.npy")

Predictions saved to labels23.npy


In [27]:
from google.colab import files

# Download the file
files.download('labels23.npy')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>